# Evaluation LIWC Swearing & insult.wiki

Meta information removed for blind review.

### 1) Prepare Evaluation Data Set

In [1]:
import pandas as pd

In [2]:
df_test = pd.read_csv('C:/Users/.../GermEval21_TestData.csv') 

In [3]:
df_test.head()

,comment_id,comment_text,Sub1_Toxic,Sub2_Engaging,Sub3_FactClaiming
0,3245,@USER Sie würden wahrscheinlich auch einen Kri...,1,1,0
1,3246,"@USER , ich glaube,Sie verkrnnen gründlich die...",0,1,1
2,3247,@USER: Man kann natürlich immerzu dieselbe Sau...,0,1,1
3,3248,"Als jemand, der im real existierenden Sozialis...",1,0,0
4,3249,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1,0,0


In [4]:
df_test = df_test[["comment_text", "Sub1_Toxic"]]
df_test.head()

,comment_text,Sub1_Toxic
0,@USER Sie würden wahrscheinlich auch einen Kri...,1
1,"@USER , ich glaube,Sie verkrnnen gründlich die...",0
2,@USER: Man kann natürlich immerzu dieselbe Sau...,0
3,"Als jemand, der im real existierenden Sozialis...",1
4,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1


In [5]:
df_test["Sub1_Toxic"].value_counts() #Derivation of Uncivil Comments in the Evaluation Data Set

0    594
1    350
Name: Sub1_Toxic, dtype: int64

## LIWC Swearing

#### Read and Prepare Dictionary

In [6]:
#Read LIWC
liwc= pd.read_table("Other Dictionaries/LIWC_German_edit", 
                    error_bad_lines=False,names=["wort",1,2,3,4,5,6,7,8,9],
                   )
liwc

,wort,1,2,3,4,5,6,7,8,9
0,ab,10,37.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN
1,abbrach*,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abbreche,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abbrich*,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abend*,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7593,zwoelf*,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7594,zwölf*,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7595,zynisch*,12,16.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN
7596,zyste*,60,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Extract LIWC Swear
liwc_swear=liwc.loc[liwc.isin([66]).any(axis=1)] #66 is the code for the sub-dictionary Swear
liwc_swear

,wort,1,2,3,4,5,6,7,8,9
126,abschaum*,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,aersche,12,16.0,18.0,60.0,61.0,62.0,66.0,NaN,NaN
212,affe,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,armleuchter*,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,arsch*,12,16.0,18.0,60.0,61.0,62.0,66.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7134,weibsstück*,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7135,weibsstueck*,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7229,wichtigtuer*,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7394,zicke*,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
liwc_swear_list = list(liwc_swear.wort)
liwc_swear_list[:10]

['abschaum*',
 'aersche',
 'affe',
 'armleuchter*',
 'arsch*',
 'ärsche',
 'bastard*',
 'bauerntoelpel*',
 'bauerntölpel*',
 'bekloppt*']

#### Function for matching

In [9]:
#Testing the matching algorithm
import regex as re

test_string = "Regierung des abschaums"

result = re.findall(liwc_swear_list[0], test_string)
print(result)
if result:
    print(result)
else:
    print("Search unsuccessful.")

['abschaum']
['abschaum']


In [10]:
liste_sterne = [] #wild card entires 
liste_nosterne = []

for e in liwc_swear_list:
        if "*" in e:
          #  print(e)
            liste_sterne.append(e)
        else:
           # print("ohne *:", e)
            liste_nosterne.append(e)
            

In [11]:
print("Number of entries with wild card:", len(liste_sterne))
liste_sterne[:10]

Number of entries with wild card: 108


['abschaum*',
 'armleuchter*',
 'arsch*',
 'bastard*',
 'bauerntoelpel*',
 'bauerntölpel*',
 'bekloppt*',
 'bescheuert*',
 'biest*',
 'blöd*']

In [12]:
print("Number of entries without wild card:", len(liste_nosterne))
liste_nosterne[:10] #wild card entires 

Number of entries without wild card: 34


['aersche',
 'affe',
 'ärsche',
 'bumse',
 'bumst',
 'durchficke',
 'durchfickst',
 'durchfickt',
 'ficke',
 'fickst']

In [13]:
from nltk.tokenize import WhitespaceTokenizer

tokenizer = WhitespaceTokenizer()#We apply Tokenization of srings by white space

def match_regex(X):
    X = str(X) 
    X = str.lower(X) #Lower string
    
    tokens = tokenizer.tokenize(X)

    match = []
    
    for i in liste_sterne: #Matching wild entires cards using findall-method
        try:
            result = re.findall(i, X)
            #print(i)
            if result:
                match.append(i)
        except:
            pass
        
    for token in tokens: #Matching non-wild card entries when token is in list of entries.
        if token in liste_nosterne:
            #print(token)
            match.append(token)
      
    
    return(match)

In [14]:
df_test["matches_liwc"]=df_test["comment_text"].apply(lambda x: match_regex(x)) #Return machtes

In [15]:
df_test["matches_count_liwc"]=df_test["matches_liwc"].apply(lambda x: len(x)) #Count number of machtes

In [16]:
df_test.head(10)

,comment_text,Sub1_Toxic,matches_liwc,matches_count_liwc
0,@USER Sie würden wahrscheinlich auch einen Kri...,1,[scheiß*],1
1,"@USER , ich glaube,Sie verkrnnen gründlich die...",0,[mist*],1
2,@USER: Man kann natürlich immerzu dieselbe Sau...,0,"[mist*, sau]",2
3,"Als jemand, der im real existierenden Sozialis...",1,"[macho*, trottel*]",2
4,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1,[],0
5,Sobald klar ist dass Trump die Wahl gewinnt li...,1,[],0
6,"@MEDIUM ...stimmt so nicht ganz: bespiel ""klei...",0,[],0
7,@USER woher wissen Sie das? Glauben heißt nich...,0,[],0
8,Warum man diesen Typen einlädt wird sich mir n...,0,[dümm*],1
9,@USER Merkel tritt nicht mehr an. Und mit Sars...,0,[arsch*],1


In [17]:
df_test["matches_count_liwc"].value_counts()

0    725
1    163
2     42
3     11
4      2
5      1
Name: matches_count_liwc, dtype: int64

In [18]:
df_test.sort_values(by=['matches_count_liwc'], ascending=False)

,comment_text,Sub1_Toxic,matches_liwc,matches_count_liwc
390,Entscheidung wurd doch korrigiert. Wo wird hie...,0,"[idiot*, scheiß*, schiß*, schiss*, vollidiot*]",5
578,"@USER es ist wahr, dass Trump keinen Krieg ang...",1,"[hintern*, idiot*, macho*, vollidiot*]",4
908,Eines muss man Trump lassen er lässt die Mensc...,0,"[hintern*, macho*, scheiß*, schiß*]",4
647,Manche Aussagen hier sind an Arroganz kaum zu ...,1,"[idiot*, macho*, mist*]",3
62,In dieser Sachlage darf man die Aussage von Ge...,0,"[dumm*, hintern*, macho*]",3
...,...,...,...,...
357,dann soll sie auch mal ihr Handy abgeben https...,0,[],0
359,Für mich ist im Hinblick auf's Auto fahren ein...,0,[],0
361,Was Klima- und Umweltschütz besonders gut könn...,1,[],0
362,"Es hieß damals, er wüsste nicht, ob er ein Pla...",0,[],0


In [19]:
df_test["liwc_dich"] = df_test["matches_count_liwc"].replace([2,3,4,5,], 1) #dichotomous coding

In [20]:
df_test["liwc_dich"].value_counts()

0    725
1    219
Name: liwc_dich, dtype: int64

#### Evaluation LIWC Swearing

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [22]:
print(classification_report(df_test.Sub1_Toxic, df_test["liwc_dich"]))

              precision    recall  f1-score   support

           0       0.66      0.80      0.72       594
           1       0.46      0.29      0.35       350

    accuracy                           0.61       944
   macro avg       0.56      0.54      0.54       944
weighted avg       0.58      0.61      0.58       944



## insult.wiki

#### Read and prepare dictionary

In [23]:
insults= pd.read_table("Other Dictionaries/insult_wiki_df.csv")

In [24]:
insults

,insult
0,Aa-Esser
1,Aa-Gesicht
2,Aa-Kopf
3,Aal
4,Aalauge
...,...
1921,Zwitter
1922,Zyklopenfotze
1923,Zyniker
1924,Zyste


In [25]:
insults_list = list(insults.insult)

In [26]:
insults_list[:10]

['Aa-Esser',
 'Aa-Gesicht',
 'Aa-Kopf',
 'Aal',
 'Aalauge',
 'Aalficker',
 'Aalfresse',
 'Aa-Loch',
 'Aalwurstverkäufer',
 'Aas']

In [27]:
insults_list = [i.lower() for i in insults_list]

In [28]:
insults_list[:10]

['aa-esser',
 'aa-gesicht',
 'aa-kopf',
 'aal',
 'aalauge',
 'aalficker',
 'aalfresse',
 'aa-loch',
 'aalwurstverkäufer',
 'aas']

In [29]:
len(set(insults_list)) #no duplicates

1926

#### Matching function using is in method

In [30]:
def match_unigrams_isin(X):
    match = []
    X = str(X)  # vorsichtshalber

    X = str.lower(X)



    for i in insults_list:
        if i in X:
            match.append(i)
           # print(i)

    return (match)

In [31]:
df_test["matches_insults"]=df_test["comment_text"].apply(lambda x: match_unigrams_isin(x))

In [32]:
df_test["matches_count_insults"] = df_test["matches_insults"].apply(lambda x: len(x))

In [33]:
df_test.head(10)

,comment_text,Sub1_Toxic,matches_liwc,matches_count_liwc,liwc_dich,matches_insults,matches_count_insults
0,@USER Sie würden wahrscheinlich auch einen Kri...,1,[scheiß*],1,1,[],0
1,"@USER , ich glaube,Sie verkrnnen gründlich die...",0,[mist*],1,1,[],0
2,@USER: Man kann natürlich immerzu dieselbe Sau...,0,"[mist*, sau]",2,1,"[ding, esel, hund, ische, sau]",5
3,"Als jemand, der im real existierenden Sozialis...",1,"[macho*, trottel*]",2,1,[trottel],1
4,Ihr größter Erfolg in ihrem Leben? KLASSENSPRE...,1,[],0,0,[],0
5,Sobald klar ist dass Trump die Wahl gewinnt li...,1,[],0,0,[sau],1
6,"@MEDIUM ...stimmt so nicht ganz: bespiel ""klei...",0,[],0,0,[],0
7,@USER woher wissen Sie das? Glauben heißt nich...,0,[],0,0,[],0
8,Warum man diesen Typen einlädt wird sich mir n...,0,[dümm*],1,1,"[affe, assi, möchtegern, nichts, niemand, typ]",6
9,@USER Merkel tritt nicht mehr an. Und mit Sars...,0,[arsch*],1,1,[],0


In [34]:
df_test.matches_count_insults.value_counts() #Overview match counts

0     661
1     177
2      65
3      20
4      17
10      1
7       1
6       1
5       1
Name: matches_count_insults, dtype: int64

In [35]:
def f(row):
    if row['matches_count_insults'] > 0:
        val = 1
    elif row['matches_count_insults'] == 0:
        val = 0
    
    return val

In [36]:
df_test['insults_dich']  = df_test.apply(f, axis=1)

In [37]:
df_test["insults_dich"].value_counts()

0    661
1    283
Name: insults_dich, dtype: int64

#### Evaluation insult.wiki

In [38]:
print(classification_report(df_test["Sub1_Toxic"], df_test["insults_dich"]))

              precision    recall  f1-score   support

           0       0.66      0.73      0.69       594
           1       0.43      0.35      0.39       350

    accuracy                           0.59       944
   macro avg       0.54      0.54      0.54       944
weighted avg       0.57      0.59      0.58       944

